## Setup and Imports

In [57]:
!nvidia-smi

Thu Feb 13 07:53:03 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   72C    P0             31W /   70W |    5782MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [58]:
!pip install -Uq git+https://github.com/huggingface/diffusers@main
!pip install -Uq git+https://github.com/huggingface/transformers@main
!pip install -Uq bitsandbytes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [59]:
from transformers import T5EncoderModel
from diffusers import (
    CogVideoXPipeline,
    CogVideoXTransformer3DModel,
    AutoencoderKLCogVideoX,
)
from diffusers.video_processor import VideoProcessor
from diffusers.utils import export_to_gif
from IPython.display import Image
import torch
import gc

## Utility to reset VRAM

In [60]:
def flush():
    torch.cuda.empty_cache()
    torch.cuda.reset_max_memory_allocated()
    torch.cuda.reset_peak_memory_stats()

## Encoding the prompt

In [61]:
model_id = "ariG23498/CogVideoX-5b-nf4-pkg"

In [62]:
flush()
gc.collect()

19023

In [63]:
text_encoder = T5EncoderModel.from_pretrained(
    model_id,
    subfolder="text_encoder",
    device_map="cpu",
)

In [64]:
pipe = CogVideoXPipeline.from_pretrained(
    "THUDM/CogVideoX-5b",
    transformer=None,
    vae=None,
    text_encoder=text_encoder,
    torch_dtype=torch.float16,
)

pipe.enable_model_cpu_offload()

Loading pipeline components...:   0%|          | 0/3 [00:00<?, ?it/s]

In [65]:
prompt = """\
A colorful, cartoon-style scene set in a bright and cheerful outdoor environment. A vibrant green parrot with expressive eyes and detailed feathers sits on a sturdy tree branch. Hanging from the same branch is a large, neatly arranged English alphabet chart with bold, easy-to-read letters. The parrot is animatedly teaching the alphabet to three young children standing below, attentively facing the tree. The kids, around 5-7 years old, are dressed in casual, playful outfits and have diverse appearances.
"""
with torch.no_grad():
    print("Encoding prompts.")
    prompt_embeds, negative_prompt_embeds = pipe.encode_prompt(
        prompt=prompt,
    )

Encoding prompts.


In [ ]:
pipe.to("cpu")

del text_encoder
del pipe

flush()
gc.collect()

## Compute latents

In [1]:
transformer = CogVideoXTransformer3DModel.from_pretrained(model_id, subfolder="transformer")

NameError: name 'CogVideoXTransformer3DModel' is not defined

In [ ]:
pipe = CogVideoXPipeline.from_pretrained(
    "THUDM/CogVideoX-5b",
    transformer=transformer,
    vae=None,
    text_encoder=None,
    tokenizer=None,
    torch_dtype=torch.float16,
)
pipe.enable_model_cpu_offload()

Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
with torch.no_grad():
    latents = pipe(
        prompt_embeds=prompt_embeds,
        negative_prompt_embeds=negative_prompt_embeds,
        guidance_scale=6,
        num_inference_steps=200,
        num_frames=16,
        output_type="latent",
    )

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
pipe.to("cpu")

del transformer
del pipe

flush()
gc.collect()

30

## Decode latents

In [ ]:
vae = AutoencoderKLCogVideoX.from_pretrained(model_id, subfolder="vae")
vae.enable_tiling()
vae.enable_slicing()

vae%2Fconfig.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/431M [00:00<?, ?B/s]

You are loading your model in 8bit or 4bit but no linear modules were found in your model. Please double check your model architecture, or submit an issue on github if you think this is a bug.


In [ ]:
latents = latents.frames

latents = latents.permute(0, 2, 1, 3, 4)  # [batch_size, num_channels, num_frames, height, width]
latents = 1 / vae.config.scaling_factor * latents

with torch.no_grad():
    frames = vae.decode(latents).sample

In [ ]:
vae_scale_factor_spatial = 2 ** (len(vae.config.block_out_channels) - 1)

video_processor = VideoProcessor(vae_scale_factor=vae_scale_factor_spatial)
video = video_processor.postprocess_video(video=frames, output_type="pil")

export_to_gif(video[0], "scenes.gif")

In [ ]:
Image(open('/kaggle/working/scenes.gif','rb').read())

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/scenes.gif'